# 📊 Pure Quantitative Trading Strategy

## Introduction

This notebook serves as a hands-on walkthrough of the end-to-end process for developing a **purely quantitative trading strategy**, inspired by the methodology outlined in *"[Still in Development.]"* The strategy design follows a well-defined, systematic framework designed to streamline the development pipeline.

As with other experiments in this repository, we leverage **Mini_lib** — our in-house research framework — to simplify and modularize key steps in quantitative research applied to financial data. This notebook is structured around the following core stages:

1. **Data Processing**  
   This stage encompasses feature extraction, labeling, sampling, and data sanitization. Its primary goal is to uncover potential sources of alpha and exploitable patterns in raw market data — essentially deciding *where to dig*.

2. **Signal Extraction**  
   Here we perform feature selection, model training, and hyperparameter tuning. This stage focuses on extracting predictive signals — *using the right tools to dig efficiently*.

3. **Strategy Tuning**  
   This includes signal filtering, position sizing, and stop-loss rules. The goal is to refine raw signals into actionable trades — *separating gold from dirt*.

4. **Risk Management**  
   This involves hedging and portfolio optimization to mitigate risk across assets and strategies — *scaling the operation and protecting against downside*.

5. **Backtesting**  
   We run simulations and stress tests to evaluate the strategy's robustness and real-world viability — *verifying if the mine is actually profitable*.

By following this structured pipeline, the notebook aims to provide a clear, modular example of how to build and validate a fully systematic trading strategy — from raw data to performance assessment.


In [1]:
import sys
sys.path.append("..")
import Data as dt
import ChevalParesseux_lib as lib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Sets the number of jobs to use for parallel processing
n_jobs = 5

## **0. Getting Data and Making Samples**  

The first step in our workflow is to **acquire and preprocess market data**. This involves:  

1. **Fetching historical price data** – Using reliable sources to obtain asset prices.  
2. **Cleaning and preprocessing** – Handling missing values, adjusting for corporate actions, and ensuring consistency.  
   *(Note: The dataset used here is already processed for daily data, so we will skip this step.)*  
3. **Creating research samples** – Splitting the data into **Training**, **Testing**, and **Embargo** sets to prevent overfitting.  

### ⚠️ **Important Note**  
A **testing set** should be used **sparingly**! It is recommended to track the number of times it is accessed to prevent data leakage and ensure unbiased model evaluation.  


In [2]:
# I. Load data
data = dt.load_data(ticker='SPY')
data['code'] = 'SPY'

In [3]:
# II. Making Samples (we use a temporal sampling here)
full_size = len(data)

training_data = data.iloc[0 : int(full_size * 0.7)].copy()
testing_data = data.loc[int(full_size * 0.7) + 1 : int(full_size * 0.9)].copy()
embargo_data = data.loc[int(full_size * 0.9) + 1 :].copy()

# III. Let's see the training data we have on a log scale
# plot_data = training_data.copy()
# plot_data.index = pd.to_datetime(plot_data['date'])

# plt.figure(figsize=(17, 5))
# plt.plot(np.log(plot_data['close']), label='Log Price', color='blue')
# plt.legend()
# plt.grid()
# plt.title('Log Prices on Training Sample')
# plt.show()

---

## **I. Data Processing**

### 🎯 **Objective**  
The objective of this stage is to prepare the dataset for effective model training by performing several key preprocessing steps:

- **Feature Extraction**: Identify and compute features that may carry predictive power.
- **Label Generation**: Create target variables if the task is supervised (classification or regression).
- **Resampling**: Adjust the training data distribution to reflect financial constraints or modeling objectives (e.g., class imbalance, volatility targeting).
- **Sanitization**: Clean and validate the extracted features and labels to ensure consistency and prevent downstream issues.

### ***1. Exploration of the data***

In [4]:
"""
The dataset used here follows a common structure for daily financial data analysis, 
containing the standard HLCV fields — High, Low, Close, and Volume.  

Notably, bid-ask spread information is not available, which means it will need to be implied or approximated 
during the backtesting phase to simulate more realistic execution conditions.
"""
training_data

,date,open,close,high,low,volume,ask_close,bid_close,ask_open,bid_open,code
0,2013-01-02,144.91,145.82,146.1575,142.398,192700020.0,145.82,145.81,144.91,144.90,SPY
1,2013-01-03,145.91,145.73,146.3700,145.340,142464794.0,145.75,145.72,145.91,145.91,SPY
2,2013-01-04,146.00,146.33,146.6100,145.670,112735212.0,146.37,146.33,146.00,145.99,SPY
3,2013-01-07,145.85,145.99,146.4500,145.430,109620278.0,145.99,145.98,145.85,145.85,SPY
4,2013-01-08,145.65,145.62,146.0370,144.980,122588765.0,145.62,145.58,145.66,145.66,SPY
...,...,...,...,...,...,...,...,...,...,...,...
2174,2021-08-20,440.23,443.36,443.7100,439.710,63589999.0,443.05,443.04,440.24,440.21,SPY
2175,2021-08-23,445.16,447.26,448.2300,445.140,49476621.0,447.52,447.50,445.17,445.16,SPY
2176,2021-08-24,447.97,447.97,448.5400,447.420,33759077.0,447.90,447.89,447.97,447.94,SPY
2177,2021-08-25,448.17,448.91,449.4570,447.770,35947463.0,448.90,448.90,448.17,448.14,SPY


In [5]:
"""
Let's make some plots to deepen our understanding of the data and validate some assumptions.

Analysis : 

- Returns distribution : Many of the financial time series are assumed to be normally distributed, however 
the empirical distribution of returns is often leptokurtic (fat-tailed) and skewed which is the case here.

We'll have to take this into account when we build our model, usually a single model will capture either the 
extreme events or the normal events, but not both.
"""
# I. Plotting the returns series distribution
# returns = training_data['close'].pct_change()

# lib.plot_series_distribution(series=returns, title='Daily Close Returns Distribution')
# lib.plot_QQ(series=returns, title='QQ Plot of Daily Close Returns Distribution')

"\nLet's make some plots to deepen our understanding of the data and validate some assumptions.\n\nAnalysis : \n\n- Returns distribution : Many of the financial time series are assumed to be normally distributed, however \nthe empirical distribution of returns is often leptokurtic (fat-tailed) and skewed which is the case here.\n\nWe'll have to take this into account when we build our model, usually a single model will capture either the \nextreme events or the normal events, but not both.\n"

In [6]:
"""
- ACF and PACF : The autocorrelation function (ACF) and partial autocorrelation function (PACF) are used to
determine the order of the ARIMA model. As we can see from the ACF and PACF plots, the returns series mostly
display no significant autocorrelation, which is a common characteristic of financial time series. Still, some
points are significant, indicating that there may be some patterns in the data that could be exploited.
"""
# II. Plotting the ACF and PACF
# lib.plot_acf(series=returns, lags=50, title='ACF of Daily Close Returns Distribution')
# lib.plot_pacf(series=returns, lags=50, title='PACF of Daily Close Returns Distribution')

'\n- ACF and PACF : The autocorrelation function (ACF) and partial autocorrelation function (PACF) are used to\ndetermine the order of the ARIMA model. As we can see from the ACF and PACF plots, the returns series mostly\ndisplay no significant autocorrelation, which is a common characteristic of financial time series. Still, some\npoints are significant, indicating that there may be some patterns in the data that could be exploited.\n'

In [7]:
"""
- Volatility clustering : The returns series exhibits volatility clustering, which is a common phenomenon in
financial time series. This means that periods of high volatility are followed by periods of low volatility and vice versa.
As we can see from the plot, the volatility of the returns series is not constant over time and tends to disminish over time.
"""
# lib.plot_volatility(series=returns, rolling_window=20, quantile=0.65, figsize=(17, 7), title='Volatility of Daily Close Returns Distribution')

'\n- Volatility clustering : The returns series exhibits volatility clustering, which is a common phenomenon in\nfinancial time series. This means that periods of high volatility are followed by periods of low volatility and vice versa.\nAs we can see from the plot, the volatility of the returns series is not constant over time and tends to disminish over time.\n'

### ***2. Labelling***

### Labeling Strategy

To maintain simplicity, we will later use a **Decision Tree Classifier** as our modeling algorithm. This choice requires the target variable to be represented as **discrete labels** rather than continuous values.

These labels will also serve a dual purpose: they will help us **analyze and interpret the predictive power** of our features.

For this purpose, we will employ the **Triple Barrier Labeling Method**, as introduced by **Marco López de Prado**. This method provides a robust framework for labeling financial time series data by considering both profit-taking and stop-loss barriers, as well as a time-based exit condition.


In [8]:
processed_train = training_data.copy()

In [9]:
# ======= I. Set up the labeller =======
labeller = lib.TripleBarrier_labeller(n_jobs=n_jobs)
labeller_params = {
    "upper_barrier": [1.5],
    "lower_barrier": [1],
    "vertical_barrier": [21],
    "vol_window": [21],
    "smoothing_method": ['ewma'],
    "window_smooth": [5],
    "lambda_smooth": [0.2],
}
labeller.set_params(**labeller_params)

# ======= II. Extract the labels =======
labels_df = labeller.extract(data=processed_train['close'])
processed_train['label'] = labels_df[labels_df.columns[0]]

In [10]:
# ======= III. Plot the labels =======
# focus_df = processed_train.iloc[1000:2000].copy()
# lib.plot_series_labels(series=focus_df['close'], label_series=focus_df['label'])

In [11]:
# ======= V. Quick backtest =======
"""
Obviously, usings labels as signals will give insane results as the labels are based on the future data.
Still, it's a nice verificatin of the labeller and the quick backtest function.
"""
# lib.plot_quick_backtest(
#     series=focus_df['close'],
#     signal=focus_df['label'],
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

"\nObviously, usings labels as signals will give insane results as the labels are based on the future data.\nStill, it's a nice verificatin of the labeller and the quick backtest function.\n"

### ***3. Features Extraction***

### Modeling Approach

To maintain simplicity in our initial model, we aim to capture the common patterns of **momentum**, as reflected in the **autocorrelation** of returns. This base model will focus on the central dynamics of return behavior.

The more extreme behaviors — particularly the **tails of the return distribution** — will be addressed separately through a **meta-model** conditioned on **volatility regimes**. This separation allows us to isolate the core signal from the higher-order risk characteristics.

In [12]:
# ======= I. Moving Average Based Feature =========
avg_feature = lib.Average_feature(n_jobs=n_jobs)
average_feature_params = {
    "window": [5, 10, 20, 50],
    "smoothing_method": [None, 'ewma'],
    "window_smooth": [5],
    "lambda_smooth": [0.2],
}
avg_feature.set_params(**average_feature_params)

average_feature_df = avg_feature.extract(data=processed_train['close'])
processed_train = pd.concat([processed_train, average_feature_df], axis=1)

In [13]:
# feature_to_plot = average_feature_df.columns[0]
# focus_df = processed_train.iloc[1000:1500].copy()

# lib.plot_series_distribution(series=focus_df[feature_to_plot], title='Smoothed Moving Average Feature Distribution')
# lib.plot_QQ(series=focus_df[feature_to_plot], title='QQ Plot of Smoothed Moving Average Feature Distribution')
# lib.plot_acf(series=focus_df[feature_to_plot], lags=50, title='ACF of Smoothed Moving Average Feature Distribution')
# lib.plot_volatility(series=focus_df[feature_to_plot], rolling_window=20, quantile=0.65, figsize=(17, 7), title='Volatility of Smoothed Moving Average Feature Distribution')

In [14]:
# ======= II. Momentum Based Feature =========
momentum_feature = lib.Momentum_feature(n_jobs=n_jobs)
momentum_feature_params = {
    "window": [5, 10, 20, 50],
    "smoothing_method": [None, 'ewma'],
    "window_smooth": [5],
    "lambda_smooth": [0.2],
}
momentum_feature.set_params(**momentum_feature_params)

momentum_feature_df = momentum_feature.extract(data=processed_train['close'])
processed_train = pd.concat([processed_train, momentum_feature_df], axis=1)

In [15]:
# ======= III. Volatility Based Feature =========
vol_feature = lib.Volatility_feature(n_jobs=n_jobs)
vol_feature_params = {
    "window": [5, 10, 20, 50],
    "smoothing_method": [None, 'ewma'],
    "window_smooth": [5],
    "lambda_smooth": [0.2],
}
vol_feature.set_params(**vol_feature_params)

vol_feature_df = vol_feature.extract(data=processed_train['close'])
processed_train = pd.concat([processed_train, vol_feature_df], axis=1)

In [16]:
# ======= IV. Plot the Features =========
# feature_to_plot_1 = average_feature_df.columns[0]
# feature_to_plot_2 = vol_feature_df.columns[0]
# feature_to_plot_3 = momentum_feature_df.columns[0]

# focus_df = processed_train.iloc[1000:1500].copy()

# print('========= Average Feature Example =========')
# lib.plot_feature_vs_label(feature_series=focus_df[feature_to_plot_1], label_series=focus_df['label'], feature_name=feature_to_plot_1)

# print('========= Volatility Feature Example =========')
# lib.plot_feature_vs_label(feature_series=focus_df[feature_to_plot_2], label_series=focus_df['label'], feature_name=feature_to_plot_2)

# print('========= Momentum Feature =========')
# lib.plot_feature_vs_label(feature_series=focus_df[feature_to_plot_3], label_series=focus_df['label'], feature_name=feature_to_plot_3)

### ***4. Sampling***

Using a **discrete labeller** such as the **triple-barrier method** introduces several **sampling biases** that must be addressed:

- **Concurrent labels**: Overlapping events distort true label uniqueness.
- **Imbalanced label distribution**: Can bias the model toward dominant classes.
- **Unequal importance of observations**: Rare but extreme events are critical to model performance due to the heavy-tailed nature of financial returns.

Even if a model performs well on average, **failing to capture rare high-impact events** can lead to **disappointing real-world performance**.

To mitigate these issues, we implement a **resampling strategy** that aims to:

- ✅ **Balance the label distribution**  
- ✅ **Maximize label uniqueness**  
- ✅ **Increase the presence of high-information points** (e.g., large absolute returns)

This ensures a more robust and representative training dataset, especially crucial in financial time series with fat tails and overlapping label signals.


In [17]:
obs_selector = lib.Temporal_uniqueness_selection(n_jobs=n_jobs, random_state=72)
obs_selector_params = {
    'label_column': ['label'],
    'price_column': ['close'],
    'n_samples': [3000],
    'replacement': [True],
    'balancing': [False],
    'vol_window': [21],
    'upper_barrier': [1.5],
    'vertical_barrier': [21],
    'grouping_column': [None],
}
obs_selector.set_params(**obs_selector_params)

In [18]:
"""
Extraction of the datasets.

The outputs may sound strange but it makes sense : The main list contains datasets for different parameters (this is done for 
multiple purposes, mainly because it allows to make montecarlo simulations to assess the sensitivity of a model to the sampling parameters).
Each sublist contains avery dataframes used (here we use a unique dataframe for daily data and a unique asset but it could be multiple
assets or intraday data split by date etc...).
"""
# 1. Extract the datasets
datasets = obs_selector.extract(data=processed_train)
resampled_df = datasets[0][0]

resampled_df = resampled_df.sort_values(by='date').copy()
resampled_df.reset_index(drop=True, inplace=True)
resampled_df

,date,open,close,high,low,volume,ask_close,bid_close,ask_open,bid_open,...,momentum_50_None_5_0.2,momentum_50_ewma_5_0.2,volatility_5_None_5_0.2,volatility_5_ewma_5_0.2,volatility_10_None_5_0.2,volatility_10_ewma_5_0.2,volatility_20_None_5_0.2,volatility_20_ewma_5_0.2,volatility_50_None_5_0.2,volatility_50_ewma_5_0.2
0,2013-05-03,160.92,161.4900,161.8889,159.7815,144467681.0,161.54,161.50,160.92,160.91,...,0.063064,0.054291,0.006635,0.001720,0.005759,0.001510,0.009409,0.003553,0.007962,0.003064
1,2013-06-21,159.24,158.8866,159.8300,157.4700,270071892.0,158.91,158.87,159.25,159.24,...,0.000987,0.021746,0.012490,0.004959,0.011586,0.003890,0.010427,0.003220,0.009149,0.003522
2,2013-07-23,169.70,169.2400,169.8700,169.0500,80226152.0,169.24,169.22,169.71,169.70,...,0.034601,0.035704,0.002781,0.000570,0.003494,0.002026,0.004351,0.002889,0.008085,0.003794
3,2013-08-02,170.28,170.8100,170.9736,170.0500,87801051.0,170.85,170.81,170.29,170.28,...,0.031088,0.020473,0.004113,0.001600,0.003319,0.001277,0.003607,0.002340,0.007922,0.003685
4,2013-08-05,170.63,170.5200,170.9882,170.3500,53567283.0,170.53,170.52,170.62,170.61,...,0.030644,0.024501,0.003806,0.001310,0.003401,0.001264,0.003644,0.002253,0.007901,0.003685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2021-08-18,442.96,439.1800,444.6250,438.9200,71729599.0,438.96,438.94,442.96,442.93,...,0.041575,0.051867,0.005644,0.002228,0.004814,0.001708,0.004993,0.002155,0.006292,0.002424
2996,2021-08-19,436.27,439.8600,441.1400,436.1200,84536469.0,440.35,440.34,436.30,436.27,...,0.038361,0.046774,0.005409,0.002395,0.004386,0.002039,0.004986,0.002335,0.006284,0.002481
2997,2021-08-20,440.23,443.3600,443.7100,439.7100,63589999.0,443.05,443.04,440.24,440.21,...,0.044896,0.045395,0.006748,0.002122,0.005042,0.001982,0.004780,0.002125,0.006341,0.002481
2998,2021-08-23,445.16,447.2600,448.2300,445.1400,49476621.0,447.52,447.50,445.17,445.16,...,0.051733,0.045995,0.007820,0.002147,0.005637,0.002040,0.005105,0.001727,0.006435,0.002488


In [19]:
"""
As we can see, the datasets are now balanced. Furthermore, we have more labels as we get close to today and each one reflect a quite important return.
"""
# 2. Overview of the datasets
values_count = resampled_df['label'].value_counts()
print(values_count)

# 3. Plot the datasets
# lib.plot_series_labels(series=resampled_df['close'], label_series=resampled_df['label']) # Whole series

label
 1.0    1213
 0.0    1023
-1.0     764
Name: count, dtype: int64


---

## **II. Signal Extraction**

### 🎯 **Objective**  
The goal of this stage is to extract trading signals from the previously processed dataset by applying a structured modeling workflow. This step transforms engineered features into actionable predictive insights.:

- **Feature Selection**: Identify and retain the most relevant features that contribute significantly to model performance..
- **Predictor Definition**: Select and define the model used to extract the signal from the data. Two broad categories can be considered: Discretionary or Machine Learning.
- **Model Tuning**: Optimize the chosen model’s hyperparameters to improve predictive performance and generalization.

### ***1. Feature Selection***

Even though neural networks are theoretically capable of approximating any function (the Universal Approximation Theorem), in practice, they often inherit biases from the data they are trained on. 

The feature selection process plays a crucial role in mitigating this issue. By carefully selecting relevant input features, we can reduce the risk of overfitting to noisy, irrelevant, or redundant data. This helps the model generalize better to unseen data, leading to improved performance and robustness.

In [20]:
# ----- 1. Feature Selection Model -----
selector = lib.Correlation_selector(n_jobs=n_jobs)
selector_params = {
    'correlation_threshold': [0.9],
}
selector.set_params(**selector_params)

# ----- 2. Fitting Model -----
non_features = ['date', 'open', 'close', 'high', 'low', 'volume', 'ask_close', 'bid_close', 'ask_open', 'bid_open', 'code', 'label']
features_df = resampled_df.drop(columns=non_features).copy()
selector.fit(data=features_df)

nb_features_dropped = len(selector.features_to_drop)
print(f'Number of features dropped : {nb_features_dropped}')
print(f'Features that share a correlation higher than the defined threshold with another one : \n{selector.features_to_drop}')

# ----- 3. Extracting the features -----
training_df = selector.extract(data=resampled_df)
training_df

Number of features dropped : 16
Features that share a correlation higher than the defined threshold with another one : 
{'volatility_20_None_5_0.2', 'momentum_10_None_5_0.2', 'momentum_50_ewma_5_0.2', 'momentum_20_ewma_5_0.2', 'average_50_ewma_5_0.2', 'average_10_None_5_0.2', 'momentum_5_ewma_5_0.2', 'momentum_20_None_5_0.2', 'momentum_10_ewma_5_0.2', 'average_10_ewma_5_0.2', 'volatility_10_None_5_0.2', 'average_20_None_5_0.2', 'momentum_5_None_5_0.2', 'average_20_ewma_5_0.2', 'volatility_20_ewma_5_0.2', 'volatility_50_ewma_5_0.2'}


,date,open,close,high,low,volume,ask_close,bid_close,ask_open,bid_open,code,label,average_5_None_5_0.2,average_5_ewma_5_0.2,average_50_None_5_0.2,momentum_50_None_5_0.2,volatility_5_None_5_0.2,volatility_5_ewma_5_0.2,volatility_10_ewma_5_0.2,volatility_50_None_5_0.2
0,2013-05-03,160.92,161.4900,161.8889,159.7815,144467681.0,161.54,161.50,160.92,160.91,SPY,1.0,-0.010837,-0.005392,-0.035201,0.063064,0.006635,0.001720,0.001510,0.007962
1,2013-06-21,159.24,158.8866,159.8300,157.4700,270071892.0,158.91,158.87,159.25,159.24,SPY,1.0,0.022687,0.010160,0.019704,0.000987,0.012490,0.004959,0.003890,0.009149
2,2013-07-23,169.70,169.2400,169.8700,169.0500,80226152.0,169.24,169.22,169.71,169.70,SPY,0.0,-0.001595,-0.003677,-0.028634,0.034601,0.002781,0.000570,0.002026,0.008085
3,2013-08-02,170.28,170.8100,170.9736,170.0500,87801051.0,170.85,170.81,170.29,170.28,SPY,-1.0,-0.007189,-0.003813,-0.034340,0.031088,0.004113,0.001600,0.001277,0.007922
4,2013-08-05,170.63,170.5200,170.9882,170.3500,53567283.0,170.53,170.52,170.62,170.61,SPY,-1.0,-0.003354,-0.004214,-0.032128,0.030644,0.003806,0.001310,0.001264,0.007901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2021-08-18,442.96,439.1800,444.6250,438.9200,71729599.0,438.96,438.94,442.96,442.93,SPY,0.0,0.011531,0.001908,-0.013226,0.041575,0.005644,0.002228,0.001708,0.006292
2996,2021-08-19,436.27,439.8600,441.1400,436.1200,84536469.0,440.35,440.34,436.30,436.27,SPY,0.0,0.007580,0.004185,-0.013923,0.038361,0.005409,0.002395,0.002039,0.006284
2997,2021-08-20,440.23,443.3600,443.7100,439.7100,63589999.0,443.05,443.04,440.24,440.21,SPY,0.0,-0.001529,0.003332,-0.020816,0.044896,0.006748,0.002122,0.001982,0.006341
2998,2021-08-23,445.16,447.2600,448.2300,445.1400,49476621.0,447.52,447.50,445.17,445.16,SPY,0.0,-0.010106,0.000084,-0.028328,0.051733,0.007820,0.002147,0.002040,0.006435


### ***2. Predictor Selection***

In this experiment, we aim to keep things simple and intuitive. That's why we start with a basic yet insightful model: a **Decision Tree Classifier**.

While Decision Trees are easy to interpret and fast to train, they often suffer from high variance and a tendency to overfit, especially when left unpruned. Additionally, they may struggle to capture complex, non-linear relationships in the data.

However, based on our current feature set, which appears to exhibit quasi-linear or regime-like behavior with respect to the target variable, a Decision Tree can still offer meaningful insights and serve as a reasonable baseline.

For improved performance and generalization, I recommend experimenting with more robust ensemble methods such as **Random Forests** or **XGBoost**, which tend to handle overfitting and complex relationships much more effectively.

In [21]:
# ----- 1. Predictor Model -----
predictor = lib.SKL_tree_classifier(n_jobs=n_jobs, random_state=72) # 72o also known as the "beer hand" in poker, a lucky number
predictor_params = {
    'criterion': 'gini',
    'max_depth': 3,
    'min_samples_split': 20,
    'min_samples_leaf': 20,
    'max_features': None,
}
predictor.set_params(**predictor_params)

# ----- 2. Fitting Model -----
non_features = ['date', 'open', 'close', 'high', 'low', 'volume', 'ask_close', 'bid_close', 'ask_open', 'bid_open', 'code', 'label']
X_train = training_df.drop(columns=non_features).copy()
y_train = training_df['label'].copy()
predictor.fit(X_train=X_train, y_train=y_train)

In [22]:
# # ----- 3. Plot the tree -----
# from sklearn.tree import plot_tree
# clf = predictor.decision_tree

# plt.figure(figsize=(17, 5))
# plot_tree(
#     clf,
#     feature_names=X_train.columns,
#     class_names=['-1', '0', '1'],
#     filled=True,
#     rounded=True,
#     fontsize=10
# )
# plt.title("Decision Tree Classifier")
# plt.show()

In [23]:
# ----- 3. Quick Backtest -----
#! WARNING : This is a quick backtest on the Training data, the results are not realistic !
temp_X = processed_train.drop(columns=non_features).copy()
temp_X = selector.extract(data=temp_X)
temp_X = temp_X.dropna(axis=0, how='any')
signals = predictor.predict(temp_X)

# lib.plot_series_labels(series=processed_train['close'],label_series=signals,)
# lib.plot_quick_backtest(
#     series=processed_train['close'],
#     signal=signals,
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

### ***3. Predictor Tuning***

This step focuses on tuning the predictor model. Nearly all machine learning models are defined by a set of **hyperparameters** that can significantly influence their performance. Finding the optimal combination of these hyperparameters is essential to maximize model effectiveness.

A simple yet effective approach is to perform a **grid search** combined with **cross-validation**. This method allows us to systematically explore different hyperparameter combinations while using cross-validation to assess how well the model generalizes to unseen data.

By doing so, we can identify the best-performing configuration and reduce the risk of overfitting or underfitting.
For this tuning we change the predictor from before : we use a Random Forest in order to have something more robust.

In [24]:
# ----- 1. Tuner Model -----
tuner = lib.Classifier_gridSearch(n_jobs=5, random_state=72)
tuner_params = {
    'random_search': False,
    'n_samples': 0, # useless as we don't use random search
}
tuner.set_params(**tuner_params)

# ----- 2. Fitting Model -----
model = lib.SKL_randomForest_classifier
grid_universe = { # We keep a simple grid universe for the example
    'n_estimators': [100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [20, 50, 100],
    'min_samples_leaf': [20, 50, 100],
    'max_features': [None, 'sqrt', 'log2'],
}
criteria = 'accuracy'

nb_observations = len(training_df)
n_folds = 3
size_fold = nb_observations // n_folds
data = []
for i in range(0, n_folds):
    start_idx = i * size_fold
    end_idx = (i + 1) * size_fold
    X_fold = training_df.iloc[start_idx : end_idx].drop(columns=non_features).copy()
    y_fold = training_df.iloc[start_idx : end_idx]['label'].copy()
    data.append((X_fold, y_fold))

tuner.fit(model=model, grid_universe=grid_universe, data=data, criteria=criteria)
best_params = tuner.best_params
print(f'With a {criteria} of {tuner.best_score:.2f}, Best parameters : {best_params}')

100%|██████████| 216/216 [00:43<00:00,  4.95it/s]


With a accuracy of 0.38, Best parameters : {'n_estimators': 100, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 100, 'min_samples_leaf': 20, 'max_features': 'sqrt'}


In [25]:
"""
The cross-validation average score is not the best metric to use here as we have a temporal data and the model is trained on the past data.
But it is still a good metric to use to assess the model performance.
"""
data_train = (X_train, y_train)
fitted_model = tuner.extract(model=model, data=data_train)

In [26]:
# ----- 3. Quick Backtest -----
"""
As we can see, the model underperfomed the underlying for most of the time but made really good trades during the 2020 crisis.
Also, the model predict mostly long trades which can be a worry if the underlying start to go on a long period bear market.
The model is not really good but it is a good start and we can improve it by tuning the parameters and adding more features.

WARNING : This is a quick backtest on the Training data which are the data the model was trained on, so it should perform worse on the testing data.
"""
signals = fitted_model.predict(temp_X)

# lib.plot_series_labels(series=processed_train['close'],label_series=signals)
# lib.plot_quick_backtest(
#     series=processed_train['close'],
#     signal=signals,
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

---

## **III. Strategy Tuning**

### 🎯 **Objective**  
The purpose of this stage is to transform raw trading signals into a fully operational strategy. This involves refining the signals, determining position sizing, and applying risk controls.

### 🧩 **Components**

1. **Signal Post-Processing**  
   Apply additional filters or transformations to improve signal quality and robustness.

2. **Position Sizing**  
   Define how much capital to allocate per signal.

3. **Stopping Rules**  
   Rules that define when to halt or exit the strategy.

### ***I. Signal Post-Processing***

This step focuses on adapting the signals to the financial environment by applying a post-processing.
For instance, here we apply a filter that activate a position only after receiving the same signal 3 times in a row, this is made in order to prevent overtrading and pay transaction costs because of false signals. However, the trade-off is that we are less reactive to quick changes in the market. 

In [27]:
# ======= I. Initializing the signal processor =======
signal_processor = lib.Confirmation_processor(n_jobs=n_jobs)
signal_processor_params = {
    'nb_confirmations': 2,
}
signal_processor.set_params(**signal_processor_params)

# ======= II. Extract processed signals =======
processed_signal = signal_processor.extract(signal_series=signals)
processed_train['signal'] = processed_signal

# ======= III. Plot the processed signals =======
# lib.plot_series_labels(series=processed_train['close'],label_series=processed_signal)
# lib.plot_quick_backtest(
#     series=processed_train['close'],
#     signal=processed_train['signal'],
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

### ***II. Sizing***

In [28]:
# ======= I. Initializing the sizer =======
sizer = lib.Volatility_sizer(n_jobs=n_jobs)
sizer_params = {
    'target_volatility': 0.2 / np.sqrt(252),
    'estimation_window': 21,
    'estimation_column': 'close',
    'minimum_change': 0.05,
    'max_leverage': 2,
}
sizer.set_params(**sizer_params)

# ======= II. Extract sizes =======
sizes = sizer.extract(data=processed_train)

In [29]:
# plt.figure(figsize=(17, 5))
# plt.plot(sizes, label='Sized Signal', color='blue')
# plt.legend()
# plt.grid()
# plt.title('Sized Signal')
# plt.show()

# #! This is still on the training data, so the results are not realistic !
# lib.plot_quick_backtest(
#     series=processed_train['close'],
#     signal=processed_train['signal'],
#     size=sizes,
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

### ***III. Stopping Rules***

In [30]:
"""
We will skip this section for now as it is not really useful for the example.
"""

'\nWe will skip this section for now as it is not really useful for the example.\n'

---
## **IV. Wrapping a Strategy**

### 🎯 **Objective**

In this section, we focus on encapsulating a trading strategy within a standardized structure. This is essential to ensure compatibility with the backtesting engine, enabling seamless evaluation and analysis. Their purpose is to output one or multiple datframes containing the asset informations and the signals computed at each time stamp.

All strategies are stored in `_04_Strategies_Library` directory of the library for better modularity and reuse, this one as well is stored in the exampels section labelled as `Simple_momentum`.

In [31]:
# ======= I. Set up Parameters =======
# 1. Computing parameters
non_features = ['date', 'open', 'close', 'high', 'low', 'volume', 'ask_close', 'bid_close', 'ask_open', 'bid_open', 'code', 'label']

# 2. Data Processing parameters
average_feature_params = {"window": [5, 10, 20, 50],"smoothing_method": [None, 'ewma'],"window_smooth": [5],"lambda_smooth": [0.2]}
momentum_feature_params = {"window": [5, 10, 20, 50], "smoothing_method": [None, 'ewma'],"window_smooth": [5], "lambda_smooth": [0.2]}
vol_feature_params = {"window": [5, 10, 20, 50], "smoothing_method": [None, 'ewma'],"window_smooth": [5],"lambda_smooth": [0.2]}
labeller_params = { "upper_barrier": [1.5],"lower_barrier": [1], "vertical_barrier": [21], "vol_window": [21],"smoothing_method": ['ewma'],"window_smooth": [5],"lambda_smooth": [0.2]}
obs_selector_params = {'label_column': ['label'],'price_column': ['close'],'n_samples': [3000],'replacement': [True],'balancing': [False],'vol_window': [21],'upper_barrier': [1.5],'vertical_barrier': [21],'grouping_column': [None]}

# 3. Strategy Tuning parameters
selector_params = {'correlation_threshold': [0.9],}
tuner_params = {
    'tuning_params': {'random_search': False,'n_samples': 0},
    'grid_universe': {'n_estimators': [100],'criterion': ['gini', 'entropy'],'max_depth': [3, 4, 5, 6],'min_samples_split': [20, 50, 100],'min_samples_leaf': [20, 50, 100],'max_features': [None, 'sqrt', 'log2'],},
    'criteria': 'accuracy',
}
signal_processor_params = {'nb_confirmations': 2}
sizer_params = {
    'target_volatility': 0.2 / np.sqrt(252),
    'estimation_window': 21,
    'estimation_column': 'close',
    'minimum_change': 0.05,
    'max_leverage': 2,
}

In [32]:
# ====== II. Initializing the strategy ======
strategy = lib.Simple_momentum(n_jobs=n_jobs)
strategy.set_params(
    non_features=non_features,
    labeller_params=labeller_params,
    average_feature_params=average_feature_params,
    momentum_feature_params=momentum_feature_params,
    vol_feature_params=vol_feature_params,
    obs_selector_params=obs_selector_params,
    selector_params=selector_params,
    tuner_params=tuner_params,
    signal_processor_params=signal_processor_params,
    sizer_params=sizer_params,
)

In [33]:
# ======= III. Fit the Strategy =======
"""
For this strategy, the signals are computed at the close price. We can then operate at the open price of the next day.
"""
strategy.fit(data=training_data)

100%|██████████| 216/216 [00:44<00:00,  4.89it/s]


In [34]:
# ======= IV. Verify strategy =======
"""
As the strategy we have defined is supposed to be strictly similar to what we explore in this notebook, we can quickly output the results of the strategy
on the training data to verify we obtain the same results.
"""
signals_df = strategy.extract(data=training_data)
# lib.plot_quick_backtest(
#     series=signal_df['close'],
#     signal=signal_df['signal'],
#     size=signal_df['size'],
#     frequence='daily',
#     title='Quick Backtest of the Labels on the Series',
#     figsize=(17, 5),
# )

---

## **IV. Risk Management**

### 🎯 **Objective**

This section focuses on portfolio management and risk control across various strategies.  
Since this notebook concentrates on a single-asset strategy, most risk management was already addressed during the **Strategy Tuning** phase.  
Thus, we will just pass through the Equal Weight Portfolio in order to still get a standardized output for the backtest to run smoothly but the weigtht will always be equal 1 as we trade only one asset.

In [35]:
"""
We define the portfolio to use for the backtest.
We use the Equalweights portfolio which is a simple portfolio that allocates the same weight to each asset in the portfolio.
As we only have one asset, it will allocate the same weight to the asset at each time step.

signals_df is a list containing a single dataframe with the signals, sizes, weight and other information.
"""
portfolio = lib.Equalweights_portfolio(n_jobs=n_jobs)
signals_dfs = portfolio.extract(data=signals_df)

---

## **VI. Backtesting**

### 🎯 **Objective**

This section focuses on backtesting strategies as close as possible to production.  

In [36]:
def plot_labels2(data_df, start_date, end_date, label_name):
    """
    Visualize the price with different colors based on labels within the given date range.

    Args: 
        data_df (pd.DataFrame): DataFrame containing the asset's price and labels.
        start_date (str): Start date of the date range.
        end_date (str): End date of the date range.
        label_name (str): Name of the column containing the labels.
    """
    asset = data_df.columns[0]
    data_df_cut = data_df.loc[start_date:end_date].copy()

    plt.figure(figsize=(17, 5))

    # Define colors for each label
    colors = {1: 'green', 0: 'black', -1: 'red'}

    # Plot the series with different colors based on labels
    prev_index = None
    for i in range(1, len(data_df_cut)):
        prev_index = data_df_cut.index[i - 1]
        curr_index = data_df_cut.index[i]
        label = data_df_cut[label_name].iloc[i]
        plt.plot([prev_index, curr_index], 
                 [data_df_cut[asset].iloc[i - 1], data_df_cut[asset].iloc[i]], 
                 color=colors[label], linewidth=2)

    # Add legend manually
    plt.plot([], [], color='green', label='Bullish Regime')
    plt.plot([], [], color='black', label='Neutral Regime')
    plt.plot([], [], color='red', label='Bearish Regime')

    plt.title(f'{asset} Price and Regimes')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(False)
    plt.show()
    

In [37]:
""""""
backtester = lib.Backtester(n_jobs=1)
backtester_params = {
    'brokerage_cost': 0.000,
    'slippage_cost': 0.000,
    'ask_name': 'ask_open',
    'bid_name': 'bid_open',
    'date_name': 'date',
}
backtester.set_params(**backtester_params)

operations_df, daily_summary_df = backtester.run_backtest(signals_dfs=signals_dfs)
operations_df

,code,trade_id,side,size,size_op,date,entry_price,exit_price,unit_pnl,portfolio_weight,returns
0,SPY,aca93dc7-625d-4a1a-8c12-dc8d8f76e0c7,1.0,1.32,1.32,2013-04-18,155.51,NaN,-0.0,1.0,-0.0
1,SPY,aca93dc7-625d-4a1a-8c12-dc8d8f76e0c7,1.0,1.32,-1.32,2013-04-29,155.51,158.58,4.0524,1.0,0.026059
2,SPY,9c7513fb-4d95-43d0-bf66-664fdcfda399,1.0,2.0,2.0,2013-05-29,165.21,NaN,-0.0,1.0,-0.0
3,SPY,9c7513fb-4d95-43d0-bf66-664fdcfda399,1.0,1.87,-0.13,2013-06-06,165.21,161.6,-0.4693,1.0,-0.002841
4,SPY,9c7513fb-4d95-43d0-bf66-664fdcfda399,1.0,1.81,-0.06,2013-06-07,165.21,163.67,-0.0924,1.0,-0.000559
...,...,...,...,...,...,...,...,...,...,...,...
466,SPY,46e78691-1226-4f1f-8ce8-b7e7fb75158e,1.0,1.88,-1.88,2021-08-11,440.008394,443.8,7.12822,1.0,0.0162
467,SPY,90d40f6f-ffb5-40a7-bf10-4c3fbb086138,-1.0,2.0,2.0,2021-08-17,444.23,NaN,-0.0,1.0,-0.0
468,SPY,3da304d3-4cfe-4eeb-88e8-6afad78f5794,1.0,2.0,2.0,2021-08-23,445.17,NaN,-0.0,1.0,-0.0
469,SPY,90d40f6f-ffb5-40a7-bf10-4c3fbb086138,-1.0,2.0,-2.0,2021-08-23,444.23,445.17,-1.88,1.0,-0.004232


In [38]:
daily_summary_df

,nb_operations,daily_returns,cumulative_pnl
date,,,
2013-04-18,1,-0.000000,1.000000
2013-04-29,1,0.026059,1.026059
2013-05-29,1,-0.000000,1.026059
2013-06-06,1,-0.002841,1.023144
2013-06-07,1,-0.000559,1.022572
...,...,...,...
2021-08-10,1,-0.000000,4.205011
2021-08-11,1,0.016200,4.273132
2021-08-17,1,-0.000000,4.273132
